#Boltzmann Machine

##Downloading the dataset

###ML-100K

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2021-08-14 13:16:26--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  28.1MB/s    in 0.2s    

2021-08-14 13:16:26 (28.1 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

###ML-1M

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2021-08-14 13:16:27--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  30.8MB/s    in 0.2s    

2021-08-14 13:16:27 (30.8 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


##Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn # for neural etworks
import torch.nn.parallel # for parallel computing
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


## Importing the dataset


In [ ]:
movies = pd.read_csv('/content/ml-1m/movies.dat', sep = '::', header = None, engine= 'python' , encoding= 'latin-1')  #movies and rating are separated by ::
users  =  pd.read_csv('/content/ml-1m/users.dat', sep = '::', header = None, engine= 'python' , encoding= 'latin-1')
ratings = pd.read_csv('/content/ml-1m/ratings.dat', sep = '::', header = None, engine= 'python' , encoding= 'latin-1')

In [ ]:
ratings.head()  #0 = users #1 = movie ids  #2 ratings (1-5) #3 timespamps we dont need it

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Preparing the training set and the test set


In [ ]:
#u1.base training set # u1.test testing set using above data
training_set = pd.read_csv('/content/ml-100k/u1.base', delimiter='\t') #same like ratings data
test_set = pd.read_csv('/content/ml-100k/u1.base', delimiter='\t')



In [ ]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [ ]:
training_set = np.array(training_set, dtype = 'int')

In [ ]:
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


creating two matrices for test and train
in this matrices m(u,i) =  will be the rating given by user 'u' for the movie 'i'. Observation in row and features in coloumns

In [ ]:
nb_users = int(max(max(training_set[:, 0]) , max(test_set[:, 0] )))
nb_movies = int(max(max(training_set[:, 1]) , max(test_set[:, 1] )))
print(nb_users, nb_movies )

943 1682


## Converting the data into an array with users in lines and movies in columns


In [ ]:
def convert(data):
  #why not 2d array instead of listlist : we are going to use pytorch
  #we will put the zero when user doe not rate the movie
  #so both the train and test matrics of 943*1682 will be of same size]
  new_data = []
  for i in range(1, 1+nb_users):
    id_movies = data[:,1][(data[:,0]==i)] #taking all the movie ids for which user has given the rating
    id_ratings = data[:,2][(data[:,0]==i)]
    ratings = np.zeros(nb_movies)
    ratings[id_movies-1] = id_ratings            # movie id starts for 1 therefore
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:
print(training_set[0]) #these are the ratignns given by the user 1 to all the movies
print(test_set[5])

[0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 1.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 5.0, 0.0, 2.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0, 2.0, 5.0, 4.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 2.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 0.0, 5.0, 1.0, 5.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 4.0, 5.0, 0.0, 2.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 4.0, 3.0, 5.0, 1.0, 3.0, 0.0, 3.0, 2.0, 0.0, 4.0, 0.0, 4.0, 3.0, 0.0, 2.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 5.0, 2.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 5.0, 3.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 5.0, 0.0, 5.0, 5.0, 4.0, 0.0,

## Converting the data into Torch tensors


In [ ]:
# rows are obervations and columns are feature
# numpy array vs pytorch tensors vs tensorflow tensors
training_set = torch.FloatTensor(training_set)# the FLoatTensor class expects the list of list
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [ ]:
#here we are just going to predict if only users like the movei or not
#
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0   # 1 and2 are the movies which users did not liek
training_set[training_set >= 3] =1 


In [ ]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0   # 1 and2 are the movies which users did not liek
test_set[test_set >= 3] =1

## Creating the architecture of the Neural Network


In [ ]:
# probablisitc graphical model # Bernouli RBM # Bernouli Sample
# creating a clas for Restricted Boltzman Machine

class RBM():
  def __init__(self, nv, nh ):  # prbabilites of visible node given the hidden node #self is object o this class it self #nv no. of visible node
    self.W = torch.randn(nh,nv)   
    self.a = torch.randn(1,nh) # first dimention corresponding to batch #probability of the hidden node given the visible and vice ersa                                              
    self.b = torch.randn(1,nv)

  # function to sample the hidden node
  def sample_h(self, x): #x visible node
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v  = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
    # thsi will return the values of hidden neuron given the values of ratings

  
  def sample_v(self,y): # x is hidden node
    wy = torch.mm(y, self.W) 
    activation = wy + self.b.expand_as(wy)
    p_v_given_h  = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)

  def train(self, v0, vk, ph0, phk):#v0 firts row, vk visible nodes after k , ph0 prob od hidden nodes initiallly #gibbs sampling
    #self.W += torch.mm( v0.t(), ph0 )   -  torch.mm(vk.t(), phk)
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()

    self.b += torch.sum((v0-vk), 0)
    self.a += torch.sum((ph0-phk), 0)



In [ ]:
nv = len(training_set[0])
nh = 100
batch_size = 100  # 1 for online learning
rbm = RBM(nv, nh)

## Training the RBM


In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch+1):
  # we ned loss functron to check the differnce between predicted and real test ratigns
  train_loss = 0 
  s = 0.0
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user:id_user+batch_size]
    v0 = training_set[id_user:id_user+batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
       # markov chain monte carlo technique gibs sampling # random walk
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      #lets update the weigth and bias
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0>=0] -vk[v0>=0]))
    s+=1
  print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))
      

epoch: 1 loss: tensor(0.3507)
epoch: 2 loss: tensor(0.2506)
epoch: 3 loss: tensor(0.2492)
epoch: 4 loss: tensor(0.2481)
epoch: 5 loss: tensor(0.2460)
epoch: 6 loss: tensor(0.2515)
epoch: 7 loss: tensor(0.2460)
epoch: 8 loss: tensor(0.2485)
epoch: 9 loss: tensor(0.2488)
epoch: 10 loss: tensor(0.2367)


## Testing the RBM


In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]  # contains the real ratings
    if len(vt[vt>=0]) > 0:
       # markov chain monte carlo technique gibs sampling # random walk #blind walk
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2373)
